In [2]:
import numpy as np 
from matplotlib import pyplot as plt 
import os
from tqdm import tqdm

from read_binary import read_binary    

In [3]:
files = sorted(os.listdir("simulation"), key=lambda x:float(x.split()[0]))

In [4]:
paths = [os.path.join("simulation", x) for x in files]

radius = 0.2
ts, rss, vss = [], [], []
for p in tqdm(paths):
    box_size,t,_,rs, vs = read_binary(p)
    ts.append(t)
    rss.append(rs)
    vss.append(vs)
ts = np.array(ts)
rss = np.array(rss)
vss = np.array(vss)

100%|█████████████████████████████████████████████████████████████████████████████| 1489/1489 [00:10<00:00, 136.19it/s]


In [ ]:
box_size

In [5]:
from mpl_toolkits.mplot3d import Axes3D

def plot_shere(ax, x, y, z, r, box_size, resolution=100,**kwargs):
    """ simple function to plot a sphere to a 3d axes """

    u = np.linspace(0, 2 * np.pi, resolution)
    v = np.linspace(0, np.pi, resolution)
    
    bx,by,bz = box_size
    
    xx = r * np.outer(np.cos(u), np.sin(v)) + x
    yy = r * np.outer(np.sin(u), np.sin(v)) + y
    zz = r * np.outer(np.ones(np.size(u)), np.cos(v)) + z
    
#     xx[xx<0] = 0
#     xx[xx>bx] = bx
    
#     yy[yy<0] = 0
#     yy[yy>by] = by
    
#     zz[zz<0] = 0
#     zz[zz>bz] = bz
    
    ind = (xx>0) & (xx<bx) & (yy>0) & (yy<by) & (zz>0) & (zz<bz)
    zz[~ind] = np.nan
    if ind.any():
        ax.plot_surface(xx, yy, zz,  rstride=4, cstride=4, **kwargs)


In [ ]:
rs = rss[9]
colors = plt.rcParams["axes.prop_cycle"][:len(rs)]

# set up the figure
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# loop through the data and plot the spheres
for pos, c in zip(rs, colors):
    for i in range(-1,2):
        for j in range(-1,2):
            for k in range(-1,2):
                plot_shere(ax, *(pos+box_size*np.array([i,j,k])), 
                           radius, box_size, edgecolor='none', color=c["color"])

# set the axes limits and show the plot
ax.set_ylim([0,1])
ax.set_xlim([0,1])
ax.set_zlim([0,1])
#     plt.savefig(os.path.join("imgs", f"{i:04d}.png"))

plt.show()


In [19]:
def job(path):
    box_size,t,_,rs, vs = read_binary(path)

    colors = plt.rcParams["axes.prop_cycle"][:len(rs)]

    # set up the figure
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # loop through the data and plot the spheres
    for pos, c in zip(rs, colors):
        for i in range(-1,2):
            for j in range(-1,2):
                for k in range(-1,2):
                    plot_shere(ax, *(pos+box_size*np.array([i,j,k])), 
                               radius, box_size, edgecolor='none', color=c["color"])

    # set the axes limits and show the plot
    ax.set_ylim([0,1])
    ax.set_xlim([0,1])
    ax.set_zlim([0,1])
    plt.savefig(os.path.join("imgs", f"{t:0.4f}.png"))
    plt.close(fig)

In [ ]:
for path in tqdm(paths):
    job(path)


In [ ]:
from multiprocessing import Pool
import time
from tqdm import *

with Pool(processes=8) as p:
    max_ = 30
    with tqdm(total=len(paths)) as pbar:
        for i, _ in enumerate(p.imap_unordered(job, paths)):
            pbar.update()



  0%|                                                                                         | 0/1489 [00:00<?, ?it/s]